In [1]:
import pandas as pd
from openai import OpenAI

# OpenAI API configuration
OPENAI_API_KEY = "YOURAPIKEY"
client = OpenAI(api_key=OPENAI_API_KEY)

# Read the CSV file
df = pd.read_csv('generatingset.csv')

# Open a text file to store the raw results
with open('expressionraw.txt', 'w', encoding='utf-8') as raw_output_file:
    for index, row in df.iterrows():
        english_expression = row['Expression en anglais']
        french_expression = row['Expression en français']
        
        prompt = (
            f"Generate 10 pairs of contextual sentences (one in English and one in French) for the following English-French expressions:\n"
            f"{english_expression};\n"
            f"For example:\n"
            f"a piece of cake; a piece of cake\n"
            f"The math test was a piece of cake, I finished it in just 20 minutes.; L'examen de maths était un jeu d'enfant, je l'ai terminé en seulement 20 minutes.\n"
            f"Each pair should be separated by a semicolon and there should be nothing else in the output."
        )

        completion = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are a skilled translator and contextual sentence generator."},
                {"role": "user", "content": prompt}
            ]
        )
        
        generated_text = completion.choices[0].message.content.strip()
        raw_output_file.write(f"Expression: {english_expression}; {french_expression}\n")
        raw_output_file.write(generated_text + "\n\n")

        # Display progress log
        print(f"Progress: {index + 1}/{len(df)} expressions processed.")

        # Save the text file every 10 expressions
        if (index + 1) % 10 == 0:
            raw_output_file.flush()

print("Raw results have been saved in 'expressionraw.txt'.")


Progress: 1/2277 expressions processed.
Progress: 2/2277 expressions processed.
Progress: 3/2277 expressions processed.
Progress: 4/2277 expressions processed.
Progress: 5/2277 expressions processed.
Progress: 6/2277 expressions processed.
Progress: 7/2277 expressions processed.
Progress: 8/2277 expressions processed.
Progress: 9/2277 expressions processed.
Progress: 10/2277 expressions processed.
Progress: 11/2277 expressions processed.
Progress: 12/2277 expressions processed.
Progress: 13/2277 expressions processed.
Progress: 14/2277 expressions processed.
Progress: 15/2277 expressions processed.
Progress: 16/2277 expressions processed.
Progress: 17/2277 expressions processed.
Progress: 18/2277 expressions processed.
Progress: 19/2277 expressions processed.
Progress: 20/2277 expressions processed.
Progress: 21/2277 expressions processed.
Progress: 22/2277 expressions processed.
Progress: 23/2277 expressions processed.
Progress: 24/2277 expressions processed.
Progress: 25/2277 express

In [16]:
import pandas as pd
import langdetect
import string

def process_files(file_list, output_file):
    data = {'eng': [], 'fr': []}
    
    print(f"Starting to process files: {file_list}")
    
    for file_name in file_list:
        print(f"Processing file: {file_name}")
        with open(file_name, 'r', encoding='utf-8') as file:
            for line_num, line in enumerate(file, start=1):
                if line.strip().lower().startswith('expression'):
                    print(f"Skipping line {line_num} in {file_name}: starts with 'expression'")
                    continue

                segments = line.split(';')
                
                if len(segments) < 2:
                    print(f"Skipping line {line_num} in {file_name}: not enough segments")
                    continue

                eng_segment = segments[-2].strip()
                fr_segment = segments[-1].strip()

                try:
                    lang_eng = langdetect.detect(eng_segment)
                    lang_fr = langdetect.detect(fr_segment)
                except langdetect.lang_detect_exception.LangDetectException:
                    print(f"Skipping line {line_num} in {file_name}: language detection failed")
                    continue

                if lang_eng == 'fr' and lang_fr == 'en':
                    eng_segment, fr_segment = fr_segment, eng_segment

                if langdetect.detect(eng_segment) == 'en' and langdetect.detect(fr_segment) == 'fr':
                    eng_segment = eng_segment.translate(str.maketrans('', '', string.punctuation.replace("'", '').replace(',', '').replace('.', '')))
                    fr_segment = fr_segment.translate(str.maketrans('', '', string.punctuation.replace("'", '').replace(',', '').replace('.', '')))

                    data['eng'].append(eng_segment)
                    data['fr'].append(fr_segment)
                
                else:
                    print(f"Skipping line {line_num} in {file_name}: not a valid English-French pair")

    df = pd.DataFrame(data)
    df.to_csv(output_file, index=False)
    
    print(f'Number of samples in the dataset: {len(df)}')
    print(f"Dataset saved to {output_file}")

file_list = ['finalouputfile1.txt', 'finalouputfile2.txt','finalouputfile3.txt','finalouputfile4.txt', 'finalouputfile5.txt', 'finalouputfile6.txt','finalouputfile7.txt']
output_file = 'FinalDB.csv'
process_files(file_list, output_file)


Skipping line 18905 in finalouputfile7.txt: not enough segments
Skipping line 18906 in finalouputfile7.txt: starts with 'expression'
Skipping line 18917 in finalouputfile7.txt: not enough segments
Skipping line 18918 in finalouputfile7.txt: starts with 'expression'
Skipping line 18920 in finalouputfile7.txt: not enough segments
Skipping line 18922 in finalouputfile7.txt: not enough segments
Skipping line 18924 in finalouputfile7.txt: not enough segments
Skipping line 18926 in finalouputfile7.txt: not enough segments
Skipping line 18928 in finalouputfile7.txt: not enough segments
Skipping line 18930 in finalouputfile7.txt: not enough segments
Skipping line 18932 in finalouputfile7.txt: not enough segments
Skipping line 18934 in finalouputfile7.txt: not enough segments
Skipping line 18936 in finalouputfile7.txt: not enough segments
Skipping line 18938 in finalouputfile7.txt: not enough segments
Skipping line 18939 in finalouputfile7.txt: starts with 'expression'
Skipping line 18942 in fi

In [52]:
# Load the DataFrame from the CSV file
df = pd.read_csv('expressions.csv')

# Randomly visualize 10 pairs of sentences
random_samples = df.sample(10)
print(random_samples)

                                                      eng  \
549     The scavenger hunt was a piece of cake for the...   
72428   The thunderstorm outside resulted in a sleeple...   
46210        I can't go out now, it's raining stair rods.   
9957     He was angry and he slammed the door as he left.   
101873  The artwork displayed in the gallery was out o...   
94647   He knew he had to do something to put things r...   
117665  I can't believe that dirtbag got away with ste...   
84285   His explanation hit the nail on the head and c...   
8142    During a crisis, leaders must keep their sangf...   
66297   His ability to adapt quickly to new situations...   

                                                       fr  
549     La chasse au trésor était un jeu d'enfant pour...  
72428   L'orage à l'extérieur a provoqué une nuit blan...  
46210   Je ne peux pas sortir maintenant, il pleut des...  
9957    Il était en colère et a claqué la porte en par...  
101873  Les œuvres d'art exp